# Comparing Data Transfer Speeds with S3 Transfer Acceleration

> This lab walks you through the steps to create an S3 Bucket to compare the speeds of Direct Upload and Transfer Accelerated Upload of a file.

### 2. Create S3 bucket

In [1]:
import logging
import boto3
from botocore.exceptions import ClientError
import os
import json
def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location,
                                    ObjectOwnership='BucketOwnerEnforced')
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [6]:
bucket_name='whizlabs-23456'
client = boto3.client('s3')
create_bucket(bucket_name=bucket_name)
response = client.list_buckets()
names=[]

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    names.append(bucket['Name'])
    print(f'  {bucket["Name"]}')

True

### 3.  Comparing your data transfer speed with and without Transfer Acceleration
> Direct Upload of 55MB file without Transfer Acceleration

In [7]:
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [8]:
filename='vpc_demo.mp4'
with open(filename, "rb") as f:
    client.upload_fileobj(f, bucket_name,filename)

client.list_objects(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'EZ3P4HBVXR5E2G1C',
  'HostId': '+7KimZ/qm1Rz9/nxY2E/7nMQFYOCAAsRWDMVgc6WgHSKKxQ/YIx0yzRgVaSLFsyOhjoihE1GaMQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '+7KimZ/qm1Rz9/nxY2E/7nMQFYOCAAsRWDMVgc6WgHSKKxQ/YIx0yzRgVaSLFsyOhjoihE1GaMQ=',
   'x-amz-request-id': 'EZ3P4HBVXR5E2G1C',
   'date': 'Wed, 18 May 2022 19:51:59 GMT',
   'x-amz-bucket-region': 'us-east-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Marker': '',
 'Contents': [{'Key': 'vpc_demo.mp4',
   'LastModified': datetime.datetime(2022, 5, 18, 19, 51, 22, tzinfo=tzutc()),
   'ETag': '"eac676f6ae773548de46dbd600feeb2f-7"',
   'Size': 56408151,
   'StorageClass': 'STANDARD',
   'Owner': {'DisplayName': 'labsorganization49',
    'ID': '48f3de68752e199890b12c7b2c9970112d03f2742cdf5d01bc3a58c4512031d4'}}],
 'Name': 'whizlabs-2346',
 'Prefix': '',
 'MaxKeys': 1000,
 'EncodingType': 'ur

### 5: Uploading 55MB file with Transfer Acceleration
> Enable Transfer Acceleration

In [ ]:
client.put_bucket_accelerate_configuration(
    Bucket=bucket_name,
    AccelerateConfiguration={
        'Status': 'Enabled'
    }
)

In [ ]:
# rename video file to vpc_demo1.mp4 and re upload noting the time
filename='vpc_demo1.mp4'
with open(filename, "rb") as f:
    client.upload_fileobj(f, bucket_name,filename)

client.list_objects(Bucket=bucket_name)